In [ ]:
#default_exp representation

In [ ]:
#export
from fastai2.vision.all import *

In [ ]:
#hide
from nbdev.showdoc import *

In [ ]:
path = untar_data(URLs.MNIST_TINY)
dls = ImageDataLoaders.from_folder(path)
learn = cnn_learner(dls, resnet18, pretrained=False)
m = learn.model

# Representation

> Functions and utilities to get representations of pytorch/fastai objects.

## TODO
* recursively obtain representations

## Model representation

First we want to represent any model as a graph. A graph consists of nodes and links, the nodes will be a collection of nn.Module's and nn.Parameter's, and the links will describe the connections between them.

In [ ]:
#export
class Representation:
    'Representation of a Model'
    def __init__(self, name, inp_shape, out_shape, modules=None, params=None, links=None, xtra=None):
        store_attr(self, 'name,inp_shape,out_shape,modules,params,links')
        self.xtra = ifnone(xtra, dict())
        
    def __repr__(self):
        return (f'{self.name} {self.inp_shape} -> {self.out_shape} (\n' +
                f'  modules: {self.modules}\n' +
                f'  params: {self.params}\n' +
                f'  links: {self.links}\n' +
                f'  xtra: {self.xtra}\n' +
                 ')')

In [ ]:
#hide
# temporal test
def _print_shapes(o, bs):
    if isinstance(o, torch.Size): return ' x '.join([str(bs)] + [str(t) for t in o[1:]])
    else: return str([_print_shapes(x, bs) for x in o])

In [ ]:
#export
@patch
def to_representation(self:nn.Module, *xb):
    "Gets a summary of `self` using `xb`"
    sample_inputs,infos = layer_info(self, *xb)
    n,bs = 64,find_bs(xb)
    name = self.__class__.__name__
    modules,params,links,xtra = L(),L(),L(),{}
    inp_shape = list(apply(lambda x:x.shape, xb)[0])
    out_shape = list(apply(lambda x:x.shape, xb)[0])
    infos = L([o for o in infos if o is not None]) #see comment in previous cell
    for i,(typ,np,trn,sz) in infos.enumerate():
        modules.append({'idx':i, 'name':typ})
    
    if isinstance(self, nn.Sequential):
        idxs = modules.map(lambda x: x['idx'])
        links = idxs[:-1].map_zipwith(lambda a,b: {'source':a, 'target':b}, idxs[1:])
    else: raise NotImplementedError()
    
    return Representation(name, inp_shape, out_shape, modules, params, links, xtra)
    
    inp_sz = _print_shapes(apply(lambda x:x.shape, xb), bs)
    res = f"{self.__class__.__name__} (Input shape: {inp_sz})\n"
    res += "=" * n + "\n"
    res += f"{'Layer (type)':<20} {'Output Shape':<20} {'Param #':<10} {'Trainable':<10}\n"
    res += "=" * n + "\n"
    ps,trn_ps = 0,0
    infos = [o for o in infos if o is not None] #see comment in previous cell
    for typ,np,trn,sz in infos:
        if sz is None: continue
        ps += np
        if trn: trn_ps += np
        res += f"{typ:<20} {_print_shapes(sz, bs)[:19]:<20} {np:<10,} {str(trn):<10}\n"
        res += "_" * n + "\n"
    res += f"\nTotal params: {ps:,}\n"
    res += f"Total trainable params: {trn_ps:,}\n"
    res += f"Total non-trainable params: {ps - trn_ps:,}\n\n"
    return Representation(name, inp_shape, out_shape, modules, params, links, xtra), PrettyString(res)

In [ ]:
#export
@patch
def to_representation(self:Learner):
    "Gets a summary of the model, optimizer and loss function."
    xb = self.dls.train.one_batch()[:self.dls.train.n_inp]
    return self.model.to_representation(*xb)
    res = self.model.summary(*xb)
    res += f"Optimizer used: {self.opt_func}\nLoss function: {self.loss_func}\n\n"
    if self.opt is not None:
        res += f"Model " + ("unfrozen\n\n" if self.opt.frozen_idx==0 else f"frozen up to parameter group number {self.opt.frozen_idx}\n\n")
    res += "Callbacks:\n" + '\n'.join(f"  - {cb}" for cb in sort_by_run(self.cbs))
    return PrettyString(res)

In [ ]:
r = learn.to_representation()
r

Sequential [64, 3, 28, 28] -> [64, 3, 28, 28] (
  modules: (#60) [{'idx': 0, 'name': 'Conv2d'},{'idx': 1, 'name': 'BatchNorm2d'},{'idx': 2, 'name': 'ReLU'},{'idx': 3, 'name': 'MaxPool2d'},{'idx': 4, 'name': 'Conv2d'},{'idx': 5, 'name': 'BatchNorm2d'},{'idx': 6, 'name': 'ReLU'},{'idx': 7, 'name': 'Conv2d'},{'idx': 8, 'name': 'BatchNorm2d'},{'idx': 9, 'name': 'Conv2d'}...]
  params: (#0) []
  links: (#59) [{'source': 0, 'target': 1},{'source': 1, 'target': 2},{'source': 2, 'target': 3},{'source': 3, 'target': 4},{'source': 4, 'target': 5},{'source': 5, 'target': 6},{'source': 6, 'target': 7},{'source': 7, 'target': 8},{'source': 8, 'target': 9},{'source': 9, 'target': 10}...]
  xtra: {}
)

In [ ]:
#export
@patch
def to_json(self:Representation):
    nodes = list(self.modules)
    links = list(self.links)
    return json.dumps({'nodes':nodes, 'links':links})

In [ ]:
r.to_json()

'{"nodes": [{"idx": 0, "name": "Conv2d"}, {"idx": 1, "name": "BatchNorm2d"}, {"idx": 2, "name": "ReLU"}, {"idx": 3, "name": "MaxPool2d"}, {"idx": 4, "name": "Conv2d"}, {"idx": 5, "name": "BatchNorm2d"}, {"idx": 6, "name": "ReLU"}, {"idx": 7, "name": "Conv2d"}, {"idx": 8, "name": "BatchNorm2d"}, {"idx": 9, "name": "Conv2d"}, {"idx": 10, "name": "BatchNorm2d"}, {"idx": 11, "name": "ReLU"}, {"idx": 12, "name": "Conv2d"}, {"idx": 13, "name": "BatchNorm2d"}, {"idx": 14, "name": "Conv2d"}, {"idx": 15, "name": "BatchNorm2d"}, {"idx": 16, "name": "ReLU"}, {"idx": 17, "name": "Conv2d"}, {"idx": 18, "name": "BatchNorm2d"}, {"idx": 19, "name": "Conv2d"}, {"idx": 20, "name": "BatchNorm2d"}, {"idx": 21, "name": "Conv2d"}, {"idx": 22, "name": "BatchNorm2d"}, {"idx": 23, "name": "ReLU"}, {"idx": 24, "name": "Conv2d"}, {"idx": 25, "name": "BatchNorm2d"}, {"idx": 26, "name": "Conv2d"}, {"idx": 27, "name": "BatchNorm2d"}, {"idx": 28, "name": "ReLU"}, {"idx": 29, "name": "Conv2d"}, {"idx": 30, "name": "B

In [ ]:
learn.summary()

Sequential (Input shape: ['64 x 3 x 28 x 28'])
Layer (type)         Output Shape         Param #    Trainable 
Conv2d               64 x 64 x 14 x 14    9,408      True      
________________________________________________________________
BatchNorm2d          64 x 64 x 14 x 14    128        True      
________________________________________________________________
ReLU                 64 x 64 x 14 x 14    0          False     
________________________________________________________________
MaxPool2d            64 x 64 x 7 x 7      0          False     
________________________________________________________________
Conv2d               64 x 64 x 7 x 7      36,864     True      
________________________________________________________________
BatchNorm2d          64 x 64 x 7 x 7      128        True      
________________________________________________________________
ReLU                 64 x 64 x 7 x 7      0          False     
___________________________________________________

## Export -

In [ ]:
#hide
from nbdev.export import notebook2script
notebook2script()

Converted 00_representation.ipynb.
Converted 01_explorer.ipynb.
Converted 10_tutorial.ipynb.
Converted index.ipynb.
